## Act as the main for the backend code

In [26]:
import sys
from pathlib import Path
from backend.controller_notebook import (
    load_data,
    initialize_planner,
    find_path,
    prepare_segments,
    load_vehicle_params,
    optimize_strategy,
)

# Add backend to sys.path
backend_path = Path("./backend").resolve()
sys.path.insert(0, str(backend_path))

# File paths
DATA_DIR = Path("./backend/data")
NODES_FILE = DATA_DIR / "nodes.json"
GRAPH_FILE = DATA_DIR / "graph.json"
CARS_FILE = DATA_DIR / "Cars.json"

# Load data
nodes, graph = load_data(NODES_FILE, GRAPH_FILE)
print("Data loaded successfully.")

# Test path planner
planner = initialize_planner(nodes, graph)
print("Path planner initialized.")


Data loaded successfully.
Path planner initialized.


-------------configuration section------------- --> The following section can be altered to test starting point and end point.


In [27]:
# Define start and end cities
# Vancouver, Prince George, Chilliwack, Merritt, Banff, Clearwater, Jasper, Edson, Calgary, and Edmonton
# Vancouver and Calgary as default. If 2 Cities are Too close, the path will result in no charging needed.
start_city = "Vancouver"
end_city = "Calgary"

# Model3LR, Cybertruck is  available
car_type = "Model3LR"

------------- end of configuration block -------------

In [ ]:
# Load nodes and graph
nodes, graph = load_data(NODES_FILE, GRAPH_FILE)
# Initialize path planner
planner = initialize_planner(nodes, graph)
# Find the shortest path
shortest_path, total_distance, _ = find_path(planner, start_city, end_city)
print("Shortest Path:", shortest_path)
print("Total Distance:", total_distance)

# Prepare segments
segment_distances = prepare_segments(shortest_path, graph)

# This section prints the list of nodes visited in the shortest path in order

Shortest Path: ['Vancouver', 'Abbotsford', 'Chilliwack', 'Hope', 'Fraser Valley B', 'Fraser Valley C', 'Lake Louise', 'Banff', 'Canmore', 'Kananaskis', 'Calgary']
Total Distance: 772.78


In [29]:
# Load vehicle parameters
vehicle_params = load_vehicle_params(CARS_FILE, car_type)


# Optimize charging strategy
optimal, traditional = optimize_strategy(segment_distances, vehicle_params)

# Print optimal strategy
optimal_strategy, total_time_optimal, _, _ = optimal
print("\nOptimal Charging Strategy:")
for step in optimal_strategy:
    if step['charge_amount'] > 0:
        print(
            f"At {step['station']}, charged from "
            f"{step['departure_soc'] - step['charge_amount']}% "
            f"to {step['departure_soc']}%, charged {step['charge_amount']}%"
        )
print(f"Total Travel Time (Optimal): {total_time_optimal:.2f} hours")

# Print traditional strategy
traditional_strategy_result, total_time_traditional, _, _ = traditional
print("\nTraditional Charging Strategy:")
for step in traditional_strategy_result:
    print(
        f"At {step['station']}, charged from "
        f"{step['departure_soc'] - step['charge_amount']}% "
        f"to {step['departure_soc']}%, charged {step['charge_amount']}%"
    )
print(f"Total Travel Time (Traditional): {total_time_traditional:.2f} hours")
time_savings = total_time_traditional - total_time_optimal
time_savings_percentage = 1 - (total_time_optimal / total_time_traditional)
print(f"\nTime Savings Compared to Traditional Fill-Up Strategy: {time_savings:.2f} hours ({time_savings_percentage:.2%} reduction)")


Segment Distances:
From Vancouver to next station, Distance: 55.0 km
From Abbotsford to next station, Distance: 34.17 km
From Chilliwack to next station, Distance: 43.04 km
From Hope to next station, Distance: 44.98 km
From Fraser Valley B to next station, Distance: 207.67 km
From Fraser Valley C to next station, Distance: 207.0 km
From Lake Louise to next station, Distance: 51.12 km
From Banff to next station, Distance: 19.43 km
From Canmore to next station, Distance: 27.96 km
From Kananaskis to next station, Distance: 82.41 km
Optimal final SoC found: 0 with minimum time: 7.9601901098901084
Reconstructed optimal path successfully.

Optimal Charging Strategy:
At Fraser Valley C, charged from 20% to 40%, charged 20%
At Lake Louise, charged from 0% to 10%, charged 10%
At Kananaskis, charged from 0% to 10%, charged 10%
Total Travel Time (Optimal): 7.96 hours

Traditional Charging Strategy:
At Fraser Valley C, charged from 20% to 100%, charged 80%
Total Travel Time (Traditional): 8.39 hou